#심층 신경망 

패션 MNIST 데이터 셋을 분류하면서 케라스로 심층 신경망을 만드는 방법을 배워보기 

In [2]:
#패션 MNIST 데이터셋을 불러오기 

from tensorflow import keras 
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
#이미지 픽셀값 0~255 범위에서 0~1사이로 변환 
#2차원배열을 1차원 배열로 펼침
#train_test_split()함수로 훈련세트, 검증세트로 나눔 

from sklearn.model_selection import train_test_split
train_scaled = train_input/ 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size =0.2, random_state = 42 
)

In [4]:
#은닉층, 출력층을 케라스의 Dense 클래스로 만들기 

#dense1은 은닉층 100개의 뉴런을 가지고 활성화함수 sigmoid, input_shape의 매개변수에서 입력의 크기 (784,)로 지정 
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

In [5]:
#심층 신경망 만들기 

model= keras.Sequential([dense1, dense2])

In [6]:
#층에 대한 유용한 정보를 얻을 수 있음 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


출력크기가 (None, 100), 첫 번째 차원은 샘플의 개수를 나타냄 

샘플의 수가 아직 정의되어 있지 않기 때문에 None이다.-> 케라스 모델의 fit()메서드에 훈련 데이터를 주입하면 이 데이터를 한 번에 모두 사용하지 않고 잘게 나누어 여러번에 걸쳐 경사하강법 단계를 수행하는 미니배치 경사 하강법을 사용한다 

따라서 샘플의 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정하였음, 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 배치 차원이라고 부름


모델 파라미터는 

784 x 100 + 100 =78,500,  

100 x 10 + 10 = 1,010 

In [7]:
#층을 추가하는 다른 방법 


model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [8]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Sequential 클래스의 객체를 만들고 이 객체의 add()메서드 호출
#층을 추가 

model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [11]:
#모델을 훈련하기 

model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.5624 - accuracy: 0.8088
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4082 - accuracy: 0.8514
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3733 - accuracy: 0.8657
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3506 - accuracy: 0.8733
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3346 - accuracy: 0.8785


훈련세트에 대한 성능을 보면 추가된 층이 성능을 향상시켰다는 것을 잘 알 수 있음 
인공 신경망에 몇 개의 층을 추가하더라도 complie()메서드와 fit()메서드의 사용법은 동일함 

In [12]:
#Flatten 클래스 

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation ='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 일렬로 펼치는 역할만 함

케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작가능 

In [14]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input /255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size = 0.2, random_state =42
)

In [15]:
model. compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5287 - accuracy: 0.8138
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3913 - accuracy: 0.8589
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3510 - accuracy: 0.8749
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3332 - accuracy: 0.8816
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3203 - accuracy: 0.8857


시그모이드 함수와 비교하면 성능이 조금 향상됨을 알 수 있었음

In [16]:
#검증 세트에서의 성능 확인 

model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 3ms/step - loss: 0.3609 - accuracy: 0.8777


[0.3609309494495392, 0.8777499794960022]

In [17]:
#ADAM 

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [18]:
 model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
               metrics='accuracy')
 model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 7s 4ms/step - loss: 0.5297 - accuracy: 0.8150
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3942 - accuracy: 0.8588
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3507 - accuracy: 0.8724
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3259 - accuracy: 0.8809
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3076 - accuracy: 0.8851


In [19]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3401 - accuracy: 0.8746


[0.34012889862060547, 0.8745833039283752]

Adam이 기본 RMSprop보다 조금 나은 성능을 보인다. 